In [21]:
import psycopg2 #for database connection
import pandas as pd 
import sqlalchemy
import numpy as np
import matplotlib.pyplot as plt # for plotting
import json

# Kruskal-Wallis analysis of variance
import scipy.stats as ss # For Kruskal-Wallis test
import scikit_posthocs as sp #For post hoc tests. 
from termcolor import colored # for coloring the print text
import warnings # to ignore plot warnings

# For ggplot
from plotnine import *

# For bolding the printed text
from termcolor import colored

In [22]:
bard = pd.read_csv("Bard_DATA_FOR_R.csv")
bard

,subject_id,subject_id_ph1,z_no_trust,z_trust,z_no_answer,z_total_people,Invalid_Domain_or_Sender,Potent_Mal_Links,Spelling_or_Grammar,Appeal_to_Greed,...,Findings,Present Cue Types,Website Description,TrustWorthiness Rating,Visual Attractiveness Rating,response,BardSensitivityRate,BardVictimizationRate,BardFalsePositiveRate,BardSpecificityRate
0,52837250,52837250,13,3,0,16,11.0,0.0,1.0,0.0,...,Nothing Phishy Here,[],The image is a screenshot of the Wikipedia pag...,Very Trustworthy,Neutral,Findings: Nothing Phishy Here\nPresent Cue Typ...,0.0,1.0,NaN,NaN
1,52837251,52837251,12,3,0,15,10.0,0.0,0.0,2.0,...,Something's Phishy,"['Invalid Domain or Sender', 'Appeal to Action...",The website has a black background with a ligh...,Very Untrustworthy,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese...",1.0,0.0,NaN,NaN
2,52837253,52837253,12,3,0,15,8.0,0.0,0.0,0.0,...,Something's Phishy,"['Invalid Domain or Sender', 'Potentially Mali...",The image is a screenshot of a website that ap...,Very Untrustworthy,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Something's Phishy"",\n ""Pre...",1.0,0.0,NaN,NaN
3,52837254,52837254,1,13,1,14,0.0,0.0,1.0,0.0,...,Nothing Phishy Here,[],The image shows a screenshot of a website call...,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,0.0,1.0,NaN,NaN
4,52837255,52837255,12,2,1,14,13.0,0.0,0.0,0.0,...,Something's Phishy,"['Invalid Domain or Sender', 'Potentially Mali...",This webpage appears to be a LinkedIn login pa...,Very Untrustworthy,Neutral,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese...",1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,52847471,52847471,6,8,1,14,0.0,0.0,5.0,0.0,...,Nothing Phishy Here,[],A website about women's clothing and lingerie....,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,NaN,NaN,0.0,1.0
1921,52847473,52847473,7,8,0,15,3.0,0.0,6.0,0.0,...,Nothing Phishy Here,[],This webpage appears to be a general website d...,Neutral,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Nothing Phishy Here"",\n ""Pres...",NaN,NaN,0.0,1.0
1922,52847474,52847474,0,15,0,15,0.0,0.0,0.0,0.0,...,Nothing Phishy Here,[],The image shows the homepage of a website call...,Trustworthy,Visually Attractive or Appealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,NaN,NaN,0.0,1.0
1923,52847476,52847476,4,11,0,15,0.0,0.0,0.0,0.0,...,Nothing Phishy Here,[],This webpage promotes a blog called Clueless F...,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,NaN,NaN,0.0,1.0


In [23]:
chatgpt = pd.read_csv("CHATGPT_DATA_FOR_R.csv")
chatgpt

,subject_id,subject_id_ph1,z_no_trust,z_trust,z_no_answer,z_total_people,Invalid_Domain_or_Sender,Potent_Mal_Links,Spelling_or_Grammar,Appeal_to_Greed,...,malicious_guess,Present_Cue_Types_Guess,Website_Description,TrustWorthiness_Rating,Visual_Attractiveness_Rating,Comments,GPTSensitivityRate,GPTVictimizationRate,GPTFalsePositiveRate,GPTSpecificityRate
0,52837250,52837250,13,3,0,16,11,0,1,0,...,Something's Phishy,Invalid Domain or Sender Potentially Malicious...,The webpage is designed to mimic the layout an...,Untrustworthy,Neutral,"```json\n{\n ""Findings"": ""Something's Phishy""...",1.0,0.0,NaN,NaN
1,52837251,52837251,12,3,0,15,10,0,0,2,...,Nothing Phishy Here,NaN,The image shows a web page with a domain parki...,Neutral,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",0.0,1.0,NaN,NaN
2,52837253,52837253,12,3,0,15,8,0,0,0,...,Something's Phishy,Invalid Domain or Sender Other Phishy Findings,The image shows a webpage purporting to be ass...,Very Untrustworthy,Neutral,"```json\n{\n ""Findings"": ""Something's Phishy""...",1.0,0.0,NaN,NaN
3,52837254,52837254,1,13,1,14,0,0,1,0,...,Nothing Phishy Here,NaN,The website appears to be a legitimate content...,Neutral,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",0.0,1.0,NaN,NaN
4,52837255,52837255,12,2,1,14,13,0,0,0,...,Something's Phishy,Invalid Domain or Sender,The image depicts a login page that is designe...,Very Untrustworthy,Neutral,"```json\n{\n ""Findings"": ""Something's Phishy""...",1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,52847471,52847471,6,8,1,14,0,0,5,0,...,Nothing Phishy Here,NaN,The image shows a website titled 'The Clothing...,Neutral,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",NaN,NaN,0.0,1.0
1921,52847473,52847473,7,8,0,15,3,0,6,0,...,Something's Phishy,Invalid Domain or Sender Other Phishy Findings,The provided image shows a webpage with a list...,Untrustworthy,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Something's Phishy""...",NaN,NaN,1.0,0.0
1922,52847474,52847474,0,15,0,15,0,0,0,0,...,Nothing Phishy Here,NaN,The image appears to be a screenshot of a webs...,Trustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",NaN,NaN,0.0,1.0
1923,52847476,52847476,4,11,0,15,0,0,0,0,...,Nothing's Phishy Here,NaN,The website appears to be a blog focused on fa...,Neutral,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing's Phishy He...",NaN,NaN,0.0,1.0


In [24]:
chatgpt = chatgpt.rename(columns={'GPTSensitivityRate': 'AISensitivityRate', 'GPTVictimizationRate': 'AIVictimizationRate', 'GPTFalsePositiveRate': 'AIFalsePositiveRate', 'GPTSpecificityRate':'AISpecificityRate'})
bard = bard.rename(columns={'BardSensitivityRate': 'AISensitivityRate', 'BardVictimizationRate': 'AIVictimizationRate', 'BardFalsePositiveRate': 'AIFalsePositiveRate', 'BardSpecificityRate':'AISpecificityRate'})
chatgpt

,subject_id,subject_id_ph1,z_no_trust,z_trust,z_no_answer,z_total_people,Invalid_Domain_or_Sender,Potent_Mal_Links,Spelling_or_Grammar,Appeal_to_Greed,...,malicious_guess,Present_Cue_Types_Guess,Website_Description,TrustWorthiness_Rating,Visual_Attractiveness_Rating,Comments,AISensitivityRate,AIVictimizationRate,AIFalsePositiveRate,AISpecificityRate
0,52837250,52837250,13,3,0,16,11,0,1,0,...,Something's Phishy,Invalid Domain or Sender Potentially Malicious...,The webpage is designed to mimic the layout an...,Untrustworthy,Neutral,"```json\n{\n ""Findings"": ""Something's Phishy""...",1.0,0.0,NaN,NaN
1,52837251,52837251,12,3,0,15,10,0,0,2,...,Nothing Phishy Here,NaN,The image shows a web page with a domain parki...,Neutral,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",0.0,1.0,NaN,NaN
2,52837253,52837253,12,3,0,15,8,0,0,0,...,Something's Phishy,Invalid Domain or Sender Other Phishy Findings,The image shows a webpage purporting to be ass...,Very Untrustworthy,Neutral,"```json\n{\n ""Findings"": ""Something's Phishy""...",1.0,0.0,NaN,NaN
3,52837254,52837254,1,13,1,14,0,0,1,0,...,Nothing Phishy Here,NaN,The website appears to be a legitimate content...,Neutral,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",0.0,1.0,NaN,NaN
4,52837255,52837255,12,2,1,14,13,0,0,0,...,Something's Phishy,Invalid Domain or Sender,The image depicts a login page that is designe...,Very Untrustworthy,Neutral,"```json\n{\n ""Findings"": ""Something's Phishy""...",1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,52847471,52847471,6,8,1,14,0,0,5,0,...,Nothing Phishy Here,NaN,The image shows a website titled 'The Clothing...,Neutral,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",NaN,NaN,0.0,1.0
1921,52847473,52847473,7,8,0,15,3,0,6,0,...,Something's Phishy,Invalid Domain or Sender Other Phishy Findings,The provided image shows a webpage with a list...,Untrustworthy,Visually Unattractive or Unappealing,"```json\n{\n ""Findings"": ""Something's Phishy""...",NaN,NaN,1.0,0.0
1922,52847474,52847474,0,15,0,15,0,0,0,0,...,Nothing Phishy Here,NaN,The image appears to be a screenshot of a webs...,Trustworthy,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing Phishy Here...",NaN,NaN,0.0,1.0
1923,52847476,52847476,4,11,0,15,0,0,0,0,...,Nothing's Phishy Here,NaN,The website appears to be a blog focused on fa...,Neutral,Visually Attractive or Appealing,"```json\n{\n ""Findings"": ""Nothing's Phishy He...",NaN,NaN,0.0,1.0


In [25]:
bard

,subject_id,subject_id_ph1,z_no_trust,z_trust,z_no_answer,z_total_people,Invalid_Domain_or_Sender,Potent_Mal_Links,Spelling_or_Grammar,Appeal_to_Greed,...,Findings,Present Cue Types,Website Description,TrustWorthiness Rating,Visual Attractiveness Rating,response,AISensitivityRate,AIVictimizationRate,AIFalsePositiveRate,AISpecificityRate
0,52837250,52837250,13,3,0,16,11.0,0.0,1.0,0.0,...,Nothing Phishy Here,[],The image is a screenshot of the Wikipedia pag...,Very Trustworthy,Neutral,Findings: Nothing Phishy Here\nPresent Cue Typ...,0.0,1.0,NaN,NaN
1,52837251,52837251,12,3,0,15,10.0,0.0,0.0,2.0,...,Something's Phishy,"['Invalid Domain or Sender', 'Appeal to Action...",The website has a black background with a ligh...,Very Untrustworthy,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese...",1.0,0.0,NaN,NaN
2,52837253,52837253,12,3,0,15,8.0,0.0,0.0,0.0,...,Something's Phishy,"['Invalid Domain or Sender', 'Potentially Mali...",The image is a screenshot of a website that ap...,Very Untrustworthy,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Something's Phishy"",\n ""Pre...",1.0,0.0,NaN,NaN
3,52837254,52837254,1,13,1,14,0.0,0.0,1.0,0.0,...,Nothing Phishy Here,[],The image shows a screenshot of a website call...,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,0.0,1.0,NaN,NaN
4,52837255,52837255,12,2,1,14,13.0,0.0,0.0,0.0,...,Something's Phishy,"['Invalid Domain or Sender', 'Potentially Mali...",This webpage appears to be a LinkedIn login pa...,Very Untrustworthy,Neutral,"{\n ""Findings"": ""Something's Phishy"",\n ""Prese...",1.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,52847471,52847471,6,8,1,14,0.0,0.0,5.0,0.0,...,Nothing Phishy Here,[],A website about women's clothing and lingerie....,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,NaN,NaN,0.0,1.0
1921,52847473,52847473,7,8,0,15,3.0,0.0,6.0,0.0,...,Nothing Phishy Here,[],This webpage appears to be a general website d...,Neutral,Visually Unattractive or Unappealing,"{\n ""Findings"": ""Nothing Phishy Here"",\n ""Pres...",NaN,NaN,0.0,1.0
1922,52847474,52847474,0,15,0,15,0.0,0.0,0.0,0.0,...,Nothing Phishy Here,[],The image shows the homepage of a website call...,Trustworthy,Visually Attractive or Appealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,NaN,NaN,0.0,1.0
1923,52847476,52847476,4,11,0,15,0.0,0.0,0.0,0.0,...,Nothing Phishy Here,[],This webpage promotes a blog called Clueless F...,Neutral,Visually Unattractive or Unappealing,Findings: Nothing Phishy Here\nPresent Cue Typ...,NaN,NaN,0.0,1.0


In [26]:
# Merge the two dataframes on 'subject_id', keeping only rows where both have data for AISensitivityRate
merged_df = pd.merge(chatgpt, bard, on='subject_id', how='inner', suffixes=('_df1', '_df2'))

# Add the AISensitivityRate columns from both dataframes only when both values are not NaN
merged_df['AISensitivityRate'] = merged_df['AISensitivityRate_df1'] + merged_df['AISensitivityRate_df2']
merged_df['AIVictimizationRate'] = merged_df['AIVictimizationRate_df1'] + merged_df['AIVictimizationRate_df2']
merged_df['AIFalsePositiveRate'] = merged_df['AIFalsePositiveRate_df1'] + merged_df['AIFalsePositiveRate_df2']
merged_df['AISpecificityRate'] = merged_df['AISpecificityRate_df1'] + merged_df['AISpecificityRate_df2']



In [28]:
merged_df.to_csv("Chatgpt_Bard_FOR_R.csv")
#sorted.to_csv("Google_sorted.csv", index=False)